# Wikiracing Agent

Wikiracing is a game in which players compete to navigate from one Wikipedia page to another using only internal link. [Full description here](https://en.wikipedia.org/wiki/Wikiracing).

The code below demonstrates how we can easily:
1. Build a fully-functioning agent using Large Language Models.
1. Validate the AI-generated results using plain Python.

# Ready, Set, Go!
#### Library imports and environment initialization

In [1]:
from functools import cache
from typing import Union

import nest_asyncio
import wikipedia
from dotenv import load_dotenv
from networkx import DiGraph, is_simple_path
from openai import AsyncAzureOpenAI
from pydantic import BaseModel
from pydantic_ai import Agent, ModelRetry, Tool
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.usage import UsageLimits
from rich import print

# Workaround for runnning Pydantic AI in notebooks
nest_asyncio.apply()

# Load model configuration and api key from environment variables
load_dotenv()

True

#### Defining the Agent's interface to query Wikipedia - i.e. the "Tool" that we will provide to the LLM to call.
As an aside the term "call" is a misnomer: The LLM is merely responding to a prompt and has no ability to directly "call" anything. What happens underneath the hood is that there's a Pythonic framework (in our case Pydantic AI) that takes a structured response from the LLM asking for the tool to be called (with arguments), then calls the tool with the arguments and calls the LLM again with the result.

In [2]:
@cache
def get_outbound_links(page_name: str) -> list[str]:
    print(f'Getting outbound links for: "{page_name}"')

    try:
        page = wikipedia.WikipediaPage(page_name, redirect=True)
        return page.links
    except wikipedia.exceptions.PageError as e:
        raise ModelRetry(str(e))

#### Pulling it all together into an Agent definition
Note the structured result models, these will be useful for later integration with Python validation code.

In [3]:
class PathFound(BaseModel):
    pages_by_title: list[str]


class NoPathFound(BaseModel):
    pass

ResultType = Union[PathFound, NoPathFound]


# Agent itself, note the typing of the 
WikiracingAgent = Agent[None, ResultType](
    model=OpenAIModel(
        model_name="gpt-4o",
        openai_client=AsyncAzureOpenAI(),
    ),
    result_type=ResultType,  # type: ignore https://ai.pydantic.dev/results/#structured-result-validation
    result_retries=3,
    system_prompt="You are a Wikipedia agent that can get outbound links from a page.",
    tools=[Tool(function=get_outbound_links)],
)

#### Validation Function
We can't *prevent* the LLM from emitting an invalid result, but we can absolutely validate the actual result via Python code (read: that we tested and decided to trust) before returning the answer to our consumers. 

In this example we're going to dip into [Graph Theory](https://en.wikipedia.org/wiki/Graph_theory) by populating a graph with the pages as nodes and links as edges, then check if the path is valid using [NetworkX](https://networkx.org/) (another excellent library).

Note that we're not immediately failing the Agent's run if the result is invalid, but instead are raising the special `ModelRetry` exception, as a signal for the agent to incorporate into the run and try to bring the chain of thought back on track.

In [4]:
@WikiracingAgent.result_validator
def validate_path(data: ResultType) -> ResultType:
    # Validate the identified solution
    if isinstance(data, PathFound):
        # Instantiate a graph
        g = DiGraph()

        # For every page in the identified solution, add all outbound links
        for current_page_title in data.pages_by_title:
            [
                g.add_edge(current_page_title, linked_title)
                for linked_title in get_outbound_links(current_page_title)
            ]

        if not is_simple_path(g, data.pages_by_title):
            raise ModelRetry("Path is not valid")

    return data

#### Top-level Python Abstraction
Note the strongly typed arguments and return value, with differentiation between success and failure.

In [5]:
def race(start: str, end: str) -> ResultType:
    return WikiracingAgent.run_sync(
        f"Find a path of pages from '{start}' to '{end}'",
         # Limit the total number of tokens used by the agent to prevent runaway costs
        usage_limits=UsageLimits(total_tokens_limit=250000),
    ).data


In [6]:
result = race("Lindsay Lohan", "Barack Obama")
print(result)

Getting outbound links for: "Lindsay Lohan"

Getting outbound links for: "Condoleezza Rice"

Getting outbound links for: "George Clooney"

Getting outbound links for: "MTV"

Getting outbound links for: "New York City"

...

Getting outbound links for: "Barack Obama"

Getting outbound links for: "Lindsay Lohan"

Getting outbound links for: "Donald Trump"

Getting outbound links for: "Barack Obama"

PathFound(pages_by_title=['Lindsay Lohan', 'Donald Trump', 'Barack Obama'])

## The result: We have a winner!

Note that in this case the LLM meandered a lot, querying pages out of order (which isn't necessarily wrong, as we haven't prescribed a linear exploration strategy), and the overhead of couple of pages twice (incurring cost and latency).

That said: The agent got to the right answer and with a strong guarantee of correctness via the validation function.

# Parting Thoughts
1. Isn't this exciting?
1. Note the strategy of shifting trust, insofar correctness, from the LLM to Python code. I'll take the task of writing 10 lines of Python to validate a structured response over the alternative ofsetting up linguistic LLM guardrails (more on those in a separate post) any day.
1. There's room for cutting down cost and latency with tighter prompting of exploration strategy, but that's a topic for another post.
1. Another open question is how will other LLM models benchmark against each other? 

Stay tuned!